# Evaluacion de rendimiento
Ahora veremos como usar los modulos `timeit` y `cProfile` para evaluar el rendimiento de las funciones introducidas en el modulo 2, en la clase **reduccion del computo redundante**. 

In [1]:
from datetime import datetime
from functools import cache

from sortedcontainers import SortedDict

In [2]:
LOGS_STORE: SortedDict = SortedDict()

LOGS_STORE[datetime(2025, 4, 21, 14, 0)] = ("ERROR", "Base de datos caída")
LOGS_STORE[datetime(2025, 4, 21, 14, 5)] = ("INFO", "Servicio reiniciado")
LOGS_STORE[datetime(2025, 4, 21, 14, 10)] = ("MESSG", "Nuevo mensaje recibido en cola")
LOGS_STORE[datetime(2025, 4, 21, 14, 15)] = ("INFO", "Inicio de etapa de extracción")
LOGS_STORE[datetime(2025, 4, 21, 14, 20)] = ("INFO", "Fin de extracción y validación")
LOGS_STORE[datetime(2025, 4, 21, 14, 25)] = ("MESSG", "Datos cargados al almacenamiento")
LOGS_STORE[datetime(2025, 4, 22, 9, 0)] = ("INFO", "Ejecución automática iniciada")
LOGS_STORE[datetime(2025, 4, 22, 9, 5)] = ("ERROR", "Fallo en transformación de datos")
LOGS_STORE[datetime(2025, 4, 22, 9, 10)] = ("INFO", "Reintento de tarea fallida")
LOGS_STORE[datetime(2025, 4, 23, 8, 30)] = ("INFO", "Pipeline ejecutado exitosamente")
LOGS_STORE[datetime(2025, 4, 23, 8, 35)] = ("MESSG", "Correo de notificación enviado")
LOGS_STORE[datetime(2025, 4, 23, 8, 40)] = ("INFO", "Monitoreo finalizado sin errores")

In [ ]:
def get_logs_between(start: datetime, end: datetime) -> list[tuple[str, str]]:
    """Obtiene todos los logs ocurridos entre dos fechas dadas.

    Args:
        start (datetime): Fecha y hora inicial desde donde buscar (inclusiva)
        end (datetime): Fecha y hora final hasta donde buscar (inclusiva)

    Returns:
        list[tuple[str, str]]: Una lista de tuplas donde cada tupla contiene:
            - Primer elemento (str): El nivel del log (ERROR, INFO, MESSG)
            - Segundo elemento (str): El mensaje descriptivo del log

    Ejemplo:
        >>> start = datetime(2025, 4, 21, 14, 0)
        >>> end = datetime(2025, 4, 21, 14, 10)
        >>> get_logs_between(start, end)
        [('ERROR', 'Base de datos caída'),
         ('INFO', 'Servicio reiniciado'),
         ('MESSG', 'Nuevo mensaje recibido en cola')]
    """
    return [
        LOGS_STORE[time_range]
        for time_range in LOGS_STORE.irange(start, end, inclusive=(True, True))
    ]


@cache
def get_logs_between_cache(start: datetime, end: datetime) -> list[tuple[str, str]]:
    """Versión en caché de get_logs_between que almacena resultados en memoria.

    Esta función utiliza LRU cache para almacenar resultados previos y retornarlos
    inmediatamente si se solicita el mismo rango de fechas nuevamente, evitando
    cálculos redundantes.

    Args:
        start (datetime): Fecha y hora inicial desde donde buscar (inclusiva)
        end (datetime): Fecha y hora final hasta donde buscar (inclusiva)

    Returns:
        list[tuple[str, str]]: Una lista de tuplas donde cada tupla contiene:
            - Primer elemento (str): El nivel del log (ERROR, INFO, MESSG)
            - Segundo elemento (str): El mensaje descriptivo del log

    Nota:
        Esta función usa un tamaño de caché ilimitado (maxsize=None). Considere
        establecer un tamaño máximo si el uso de memoria es una preocupación.
    """
    return get_logs_between(start, end)

## Uso de `timeit`
En el modulo 2, en la clase **reduccion del computo redundante** vimos que hago uso del modulo `time`. ¿Cuando usar `time` y cuando `timeit`?

| Modulo | Ideal para... | Precision |
|--------|---------------|-----------|
| `time` | Medir el tiempo de una ejecucion | Media |
| `timeit` | Comparar rendimiento de codigo | Alta |

### 🧵 En resumen:  
`timeit` se usa cuando se quiere medir de forma precisa fragmentos de codigo, ya que este modulo ejecuta el codigo muchas veces y da un tiempo promedio de ejecucion. Por su parte, `time` es util cuando se quiere hallar cuanto tarde un bloque de codigo en ejecutarse una sola vez.   

In [ ]:
from timeit import timeit

In [16]:
setup = """
from datetime import datetime
import random
from __main__ import get_logs_between

start = datetime(2025, 4, 1)
end = datetime(2025, 4, 30)
"""

time = timeit(
    "get_logs_between", setup=setup, number=1000000
)  # number controla el numero total de veces que se ejecutara la funcion.
print(f"Tiempo promedio: {time:.4f} segundos")

Tiempo promedio: 0.0112 segundos


In [ ]:
setup = """
from datetime import datetime
import random
from __main__ import get_logs_between_cache

start = datetime(2025, 4, 1)
end = datetime(2025, 4, 30)
"""

time = timeit("get_logs_between_cache", setup=setup, number=1000000)
print(f"Tiempo promedio: {time:.4f} segundos")

Tiempo promedio: 0.0095 segundos


## Uso de `cProfile`
`cProfile` hace profiling, scanea tu codigo, para poder determinar varias cosas: cuanto tiempo pasa el programa en cada funcion, cuantas veces se llama, y donde se esta gastando el tiempo. 

En el modulo 2, en la clase **reduccion del computo redundante** vimos que hago uso del modulo `time`. ¿Cuando usar `time` y cuando `cProfile`?

| Modulo | Ideal para... | Precision |
|--------|---------------|-----------|
| `time` | Medir el tiempo de una ejecución | Media |
| `cProfile` | Analizar cuellos de botella y llamadas a funciones | Alta |

### 🧵 En resumen
`time` es útil para medir cuánto tarda un bloque de código en ejecutarse una sola vez, ideal para casos simples y rápidos de probar. Por su parte, `cProfile` es la mejor opción cuando necesitas entender a fondo el rendimiento de todo tu programa, ver qué funciones son las más costosas, cuántas veces se llaman y dónde se está gastando el tiempo.

In [19]:
import cProfile
from datetime import datetime

In [ ]:
start: datetime = datetime(2025, 4, 1)
end: datetime = datetime(2025, 4, 30)

In [ ]:
cProfile.run("get_logs_between(start, end)", sort="cumtime")

         13 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 838502535.py:1(get_logs_between)
        1    0.000    0.000    0.000    0.000 sortedlist.py:1072(irange)
        1    0.000    0.000    0.000    0.000 838502535.py:21(<listcomp>)
        1    0.000    0.000    0.000    0.000 sortedlist.py:1008(_islice)
        2    0.000    0.000    0.000    0.000 {built-in method _bisect.bisect_left}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {built-in method _bisect.bisect_right}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [22]:
cProfile.run("get_logs_between_cache(start, end)", sort="cumtime")

         14 function calls in 0.000 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 838502535.py:26(get_logs_between_cache)
        1    0.000    0.000    0.000    0.000 838502535.py:1(get_logs_between)
        1    0.000    0.000    0.000    0.000 sortedlist.py:1072(irange)
        1    0.000    0.000    0.000    0.000 838502535.py:21(<listcomp>)
        1    0.000    0.000    0.000    0.000 sortedlist.py:1008(_islice)
        2    0.000    0.000    0.000    0.000 {built-in method _bisect.bisect_left}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {built-in method _bisect.bisect_right}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lspro

Lastimosamente para este ejemplo no nos dice mucho, ya que la funcion para solo un rango de fechas es bastante rapida. Sin embargo en el proximo notebook y sobre todo, en el ejemplo integrador, veremos lo poderoso que puede ser usar esta herramienta.

## `time` vs `timeit` vs `cProfile`
Finalmente te dejo con esta tabla, donde se hace un VS entre `time`, `timeit` y `cProfile`.

| Modulo | Ideal para... | Precision |
|--------|---------------|-----------|
| `time` | Medir el tiempo de una ejecución | Media |
| `timeit` | Comparar rendimiento de codigo | Alta |
| `cProfile` | Analizar cuellos de botella y llamadas a funciones | Alta |